<a href="https://colab.research.google.com/github/JD950/NoSQL/blob/main/Py_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating a Graph Database

In [3]:
!pip install neo4j

In [5]:
#Deployment at https://sandbox.neo4j.com/ 

import neo4j
from neo4j import GraphDatabase
# Correct and Secure practice is to save these values as secrets, environment variables or as config file. But since we are in noteboo -- lets hardcode it for now
uri = 'bolt://44.193.24.89:7687'   

user = 'neo4j'

password = 'punctures-tan-orifices'
driver =  GraphDatabase.driver(uri, auth=(user, password))

In [6]:
# now that we know there is no syntax error, let's define the connection properly as a class
# otherwise we will have to worry about open ended connections 

class Driver:
    def __init__(this, uri, user, password):
        this.driver = GraphDatabase.driver(uri, auth=(user, password))
    def close(this):
        this.driver.close()
# __init__ is the constructor of the class, referred by the THIS pointer

In [7]:
# now let's update our class to execute queries for us
class Driver:
    def __init__(this, uri, user, password):
      this.driver = GraphDatabase.driver(uri, auth=(user, password))
    def close(this):
      this.driver.close()
      
    def executequery(this, query, parameters=None, db=None):
      session = this.driver.session(database=db) if db is not None else this.driver.session()
      response = []
      try:
        response = list(session.run(query, parameters))
      except Exception as e:
        print("Query failed:", e)
      finally: 
        if session is not None:
          session.close()
        return response




In [ ]:
# 4 ways of creating NODES:
# ()                  //anonymous node (no label or variable) can refer to any node in the database
# (p:Person)          //using variable p and label Person
# (:Technology)       //no variable, label Technology
# (work:Company)      //using variable work and label Company

In [8]:
query = 'CREATE (t:trainer)-[:EATS]->(f:food)'

mydriver = Driver(uri, user, password)

In [ ]:
mydriver.executequery(query)

In [ ]:
query = 'CREATE (t:trainer {name: "Deepanshu"})-[rel:LIKES]->(f:food {type: "pasta"})'
mydriver.executequery(query)

In [12]:
trainers = ['Deepanshu', 'Batman', 'Goofy']
food = ['pasta', 'bangers  bash ', 'cheese']
for c in range(3):
  query = 'CREATE (t:trainer {name: "'+trainers[c]+'"})-[rel:LIKES]->(f:food {type:"'+food[c]+'"})'
  mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer) RETURN t'
mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer {name: "Batman"})-[:LIKES]->(f:food) RETURN f'
mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer)-[:LIKES]->(f:food {type: "pasta"}) RETURN t'
mydriver.executequery(query)

In [16]:
trainers = ['Deepanshu', 'Batman', 'Goofy','Deepanshu', 'Batman', 'Goofy']
students = ['humans', 'monkeys', 'monkeys', 'pikachus', 'girraffe', 'cats',]
for c in range(6):
  query = 'CREATE (t:trainer {name: "'+trainers[c]+'"})-[rel:TEACHES]->(s:student {gene:"'+students[c]+'"})'
  mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer) -[:TEACHES]->(s:student) RETURN t'
mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer {name: "Batman"}) SET t.birthdate = date("1980-01-01") RETURN t'
mydriver.executequery(query)
query = 'MATCH (t:trainer) RETURN t'
mydriver.executequery(query)

In [19]:
query = 'MATCH (t:trainer)-[tea:TEACHES]-(s:student) RETURN t.name,tea.id,s.gene'
results = mydriver.executequery(query)

In [ ]:
results

In [ ]:
# Where clause
query='MATCH (d:trainer) WHERE d.name = "Deepanshu" RETURN d'
mydriver.executequery(query)

In [ ]:
query = 'MATCH (d:trainer) WHERE NOT d.name = "Deepanshu" RETURN d.name, d.birthdate'
mydriver.executequery(query)
# as we can see, d.birthdate is None for values that didnt have the property. SO we should find a way to eliminate such records

In [ ]:
query = 'MATCH (d:trainer) WHERE exists(d.birthdate) RETURN d.name, d.birthdate'
mydriver.executequery(query)
# this query selected only the Nodes where the birthdate property existed 

In [ ]:
query = 'MATCH (d:trainer)-[rel:LIKES]->(f:food) WHERE exists(d.birthdate) RETURN d.name, d.birthdate, f.type'
mydriver.executequery(query)

In [25]:
# setting some numerical properties randomly to do math ops
trainers = ['Deepanshu', 'Batman', 'Goofy']
import random
def sendRandomNumber():
  return int(random.random()*1000)

for trainer in trainers:
  query = 'MATCH (t:trainer {name: "'+trainer+'"}) SET t.age = '+str(sendRandomNumber())+' RETURN t'
  mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer) RETURN t'
mydriver.executequery(query)

In [ ]:
query = 'MATCH (t:trainer) WHERE 550 <= t.age <= 800 RETURN t'
mydriver.executequery(query)

In [ ]:
# fuzzy searches
query = 'MATCH (t:trainer) WHERE t.name STARTS WITH "G" RETURN t.name'
print(mydriver.executequery(query))
query = 'MATCH (t:trainer) WHERE t.name CONTAINS "a" RETURN t.name'
print(mydriver.executequery(query))
query = 'MATCH (t:trainer) WHERE t.name ENDS WITH "u" RETURN t.name'
print(mydriver.executequery(query))


In [ ]:
query = 'MATCH (t:trainer) WHERE t.name =~ "Go.*" RETURN t.name'
print(mydriver.executequery(query))
# . (single character)
# * (wild card)
# .* -> single characters any number of time
# Go.* -> Go followed by any number of characters

In [ ]:
query = 'MATCH (t:trainer) WHERE t.name =~ ".*an.*" RETURN t.name'
print(mydriver.executequery(query))

# .*an.* -> characters, an present, more characters 

In [ ]:
query = 'MATCH (t:trainer)-[tea:TEACHES]-(s:student) WHERE exists((:trainer)-[:LIKES]->(:food)) RETURN t.name,s.gene'
mydriver.executequery(query)

In [ ]:
# inbuilt functions
query = 'CALL dbms.functions()'
mydriver.executequery(query)